# Improved Calculators
**CS1302 Introduction to Computer Programming**
___

In [ ]:
%reset -f
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
import sys, io
import math
from math import log, exp, sin, cos, tan, pi

In this notebook, we will improve the calculators in the previous lab.

To get started, the following is a calculator that stays on until 
- you interrupt the kernel or 
- enter an invalid expression.

In [ ]:
while True: print(eval(input('>') or '"Enter something please."'))

## Quadratic Equation under Different Cases

### Zero Discriminant (Demo)

Recall from the previous lab that for the quadratic equation 
$$ax^2+bx+c=0$$
where $a$, $b$, and $c$ are real-valued coefficients, and $x$ is the unknown variable, the roots are given by
$$\frac{-b-\sqrt{b^2-4ac}}{2a}, \frac{-b+\sqrt{b^2-4ac}}{2a}.$$

The roots are the same (repeated) when the discriminant $b^2-4ac$ is zero.

**How to return only one root when the discriminant is zero?**

We can use the [`if` statement](https://docs.python.org/3/reference/compound_stmts.html#if).

In [ ]:
def get_roots(a, b, c):
    d = b**2 - 4 * a * c    # discriminant
    if math.isclose(d, 0):
        roots = -b / 2 / a  # repeated root
    else:
        d **= 0.5           # *
        roots = -(b - d) / 2 / a, -(b + d) / 2 / a
    return roots

In [ ]:
# quadratic equations solver
@interact(a=(-10,10,1),b=(-10,10,1),c=(-10,10,1))
def quadratic_equation_solver(a=1,b=2,c=1):
    print('Root(s):',get_roots(a,b,c))

**Why use `math.isclose(d,0)` instead of `d == 0`?**

Since floating point numbers are not *precise*, `d == 0` may be false due to finite precision error even when the discriminant is supposed to be zero.

<b>Can we move the line `d**0.5` before the if statement?</b>

`math.isclose` can take as arguments integers or floats but not complex numbers.  
However, `d**0.5` can be a complex number if `d < 0`.

### Zero coefficients

If $a=0$, the earlier formula for the roots are invalid due to division by (zero) $a$. Nevertheless, the equation remain valid:
$$bx + c=0.$$

**Exercise** Improve the function `get_roots` to return root(s) under all cases:
- If $a=0$ and $b\neq 0$, assign `roots` to the single root $-\frac{c}{b}$. 
- If $a=b=0$ and $c\neq 0$, assign `roots` to `None`.  
    Note that `None` is an object, not a string.
- If $a=b=c=0$, there are infinitely many roots. Assign to `roots` the tuple `-float('inf'), float('inf')`.  
    Note that `float('inf')` converts the string `'inf'` to a floating point value that represents $\infty$.

*Hint:* Use nested `if` statements such as the followings (with the blanks filled in properly):
```Python
def get_roots(a, b, c):
    d = b**2 - 4 * a * c
    if __________________:
        if __________________:
            if __________________:
                roots = -float('inf'), float('inf')
            else:
                roots = None
        else:
            roots = -c / b
    elif math.isclose(d, 0):
        roots = -b / 2 / a
    else:
        d **= 0.5
        roots = -(b - d) / 2 / a, -(b + d) / 2 / a
    return roots
```

In [ ]:
def get_roots(a, b, c):
    d = b**2 - 4 * a * c
    # YOUR CODE HERE
    raise NotImplementedError()
    elif math.isclose(d, 0):
        roots = -b / 2 / a
    else:
        d **= 0.5
        roots = -(b - d) / 2 / a, -(b + d) / 2 / a
    return roots

In [ ]:
# tests
def test_get_roots(roots, a, b, c):
    roots_ = get_roots(a, b, c)
    if roots is None:
        correct = roots_ is None
    elif isinstance(roots, float):
        correct = isinstance(roots_, float) and math.isclose(roots, roots_)
    else:
        correct = isinstance(roots_, tuple) and len(roots_) == 2 and all([
            math.isclose(root, roots_) for root, roots_ in zip(roots, roots_)
        ])
    if not correct:
        print(f'With (a, b, c)={a,b,c}, roots should be {roots} not {roots_}.')
    assert correct


test_get_roots((-0.0, -1.0), 1, 1, 0)
test_get_roots(0.0, 1, 0, 0)
test_get_roots((-float('inf'), float('inf')), 0, 0, 0)
test_get_roots(None, 0, 0, 1)
test_get_roots(0.5, 0, -2, 1)
test_get_roots(1.0, 1, -2, 1)

In [ ]:
# quadratic/linear equations solver
@interact(a=(-10,10,1),b=(-10,10,1),c=(-10,10,1))
def quadratic_linear_equation_solver(a=0,b=2,c=1):
    print('Root(s):',get_roots(a,b,c))

## Conversion of Arbitrarily Large Numbers

### Binary-to-Decimal (Demo)

In the previous lab, we consider converting a byte string to decimal.  
What about converting a binary string of arbitrary length to decimal?

Given a binary string of an arbitrarily length $k$,
$$ b_{k-1}\circ \dots \circ b_1\circ b_0, $$
the decimal number can be computed by the formula
$$2^0 \cdot b_0 + 2^1 \cdot b_1 + \dots + 2^{k-1} \cdot b_{k-1}.$$

In mathematics, we use the summation notation to write the above formula:
$$ \sum_{i=0}^{k-1} 2^i \cdot b_{i} $$

The formula can be implemented as a for loop: 

In [ ]:
def binary_to_decimal(binary_str):
    k = len(binary_str)
    decimal = 0                                     # initialization
    for i in range(k):
        decimal += 2**i * int(binary_str[(k-1)-i])  # iteration
    return decimal

In [ ]:
# binary-to-decimal converter
bits = ['0', '1']
@interact(binary_str='1011')
def convert_byte_to_decimal(binary_str):
    for bit in binary_str:
        if bit not in bits:
            print('Not a binary string.')
            break
    else:
        print('decimal:', binary_to_decimal(binary_str))

Note that $b_i$ is given by `binary_str[(k-1)-i]`:
$$\begin{array}{c|c:c:c:c|}\texttt{binary_str} & b_{k-1} & b_{k-2} & \dots &1\\ \text{indexing} & [0] & [1] & \dots & [k-1] \end{array}$$

The following is a better way to write the for loop without
- the possibly confusing string indexing `binary_str[(k-1)-i]`, nor
- the exponentiation `2**i`.

In [ ]:
def binary_to_decimal(binary_str):
    decimal = 0                                # initialization
    for bit in binary_str:
        decimal = decimal * 2 + int(bit)       # iteration
    return decimal

The algorithm implements the same formula factorized as follows:
$$\begin{aligned} \sum_{i=0}^{k-1} 2^i \cdot b_{i} &=  \left(\sum_{i=1}^{k-1} 2^{i-1} \cdot b_{i}\right)\times 2 + b_0 \\
&= \underbrace{(\dots (\underbrace{(\underbrace{\overbrace{0}^{\text{initialization}\kern-2em}\times 2 + b_{k-1}}_{\text{first iteration} }) \times 2 + b_{k-2}}_{\text{second iteration} }) \dots )\times 2 + b_0}_{\text{last iteration} }.\end{aligned}$$ 

### Undecimal-to-Decimal

A base-11 number system is called an [undecimal system](https://en.wikipedia.org/wiki/Undecimal). The digits range from 0 to 10 with 10 denoted as X:
$$0, 1, 2, 3, 4, 5, 6, 7, 8, 9, X.$$

The [International Standard Book Number (ISBN)](https://en.wikipedia.org/wiki/International_Standard_Book_Number) as well as your Lab assignment number both uses a undecimal digit.

**Exercise** In the following code, assign to `decimal` the integer represented by an undecimal string of arbitrary length.  
*Hint:* Write a conditional to 
1. check if a digit is (capital) `'X'`, and if so, 
2. convert the digit to the integer value 10.

In [ ]:
def undecimal_to_decimal(undecimal_str):
    decimal = 0
    for undecimal in undecimal_str:
        # YOUR CODE HERE
        raise NotImplementedError()
    return decimal

In [ ]:
# tests
def test_undecimal_to_decimal(decimal, undecimal_str):
    decimal_ = undecimal_to_decimal(undecimal_str)
    correct = isinstance(decimal_, int) and decimal_ == decimal
    if not correct:
        print(f'{undecimal_str} should give {decimal} not {decimal_}.')
    assert correct


test_undecimal_to_decimal(27558279079916281, '6662X0X584839464')
test_undecimal_to_decimal(23022771839270, '73769X2556695')
test_undecimal_to_decimal(161804347284488, '476129248X2067')

In [ ]:
# undecimal-to-decimal calculator
undecimal_digits = [str(i) for i in range(10)] + ['X']
@interact(undecimal_str='X')
def convert_undecimal_to_decimal(undecimal_str):
    for digit in undecimal_str:
        if digit not in undecimal_digits:
            print('Not an undecimal string.')
            break
    else:
        print('decimal:', undecimal_to_decimal(undecimal_str))

### Decimal-to-Binary (Demo)

Consider the reverse process that converts a non-negative decimal number of arbitrary size to a string representation in another number system.

In particular, how to convert to binary?

In [ ]:
def decimal_to_binary(decimal):
    binary_str = str(decimal % 2)
    while decimal // 2:
        decimal //= 2
        binary_str = str(decimal % 2) + binary_str
    return binary_str

In [ ]:
# decimal-to-binary calculator
@interact(decimal='11')
def convert_decimal_to_binary(decimal):
    if not decimal.isdigit():
        print('Not a non-negative integer.')
    else:
        print('binary:', decimal_to_binary(int(decimal)))

To understand the while loop, consider the same formula before, where the braces indicate the value of `decimal` at different time:
$$\begin{aligned} \sum_{i=0}^{k-1} 2^i \cdot b_{i} &=  \left(\sum_{i=1}^{k-1} 2^{i-1} \cdot b_{i}\right)\times 2 + b_0 \\
&= \underbrace{(\underbrace{ \dots (\underbrace{(0\times 2 + b_{k-1}) \times 2 + b_{k-1}}_{\text{right before the last iteration} }  )\times 2 \dots + b_1}_{\text{right before the second iteration} })\times 2 + b_0}_{\text{right before the first iteration} }.\end{aligned}$$ 

- $b_0$ is the remainder `decimal % 2` right before the first iteration,
- $b_1$ is the remainder `decimal // 2 % 2` right before the second iteration, and
- $b_{k-1}$ is the remainder `decimal // 2 % 2` right before the last iteration.

We can also write a for loop instead of a while loop but doing so requires an extra calculation of the number of iterations.

### Decimal-to-Undecimal

**Exercise** Assign to `undecimal_str` the undecimal string that represents a non-negative integer `decimal` of any size.

*Hint:* The following is a solution template but you need not follow it exactly:
```Python
def decimal_to_undecimal(decimal):
    undecimal_str = '' # last digit will be obtained in the loop as well
    while True:        # to avoid code duplication
        digit_value = ____________
        undecimal_str = (___ if ___________ == 10 else
                         str(___________)) + undecimal_str
        decimal //= 11
        if not decimal: _____
    return undecimal_str
```

In [ ]:
def decimal_to_undecimal(decimal):
    # YOUR CODE HERE
    raise NotImplementedError()
    return undecimal_str

In [ ]:
# tests
def test_decimal_to_undecimal(undecimal,decimal):
    undecimal_ = decimal_to_undecimal(decimal)
    correct = isinstance(undecimal, str) and undecimal == undecimal_
    if not correct:
        print(
            f'{decimal} should be represented as the undecimal {undecimal}, not {undecimal_}.'
        )
    assert correct

test_decimal_to_undecimal('0', 0)
test_decimal_to_undecimal('X', 10)
test_decimal_to_undecimal('1752572309X478', 57983478668530)

In [ ]:
# undecimal-to-decimal calculator
@interact(decimal='10')
def convert_decimal_to_undecimal(decimal):
    if not decimal.isdigit():
        print('Not a non-negative integer.')
    else:
        print('undecimal:', decimal_to_undecimal(int(decimal)))